In [ ]:
# | default_exp _components.producer_decorator

In [ ]:
# | export

import functools
import json
from asyncio import iscoroutinefunction  # do not use the version from inspect
from collections import namedtuple
from typing import *

from aiokafka import AIOKafkaProducer
from pydantic import BaseModel

In [ ]:
import asyncio

from pydantic import Field

from fastkafka._components.aiokafka_producer_manager import \
    AIOKafkaProducerManager
from fastkafka._testing.local_broker import LocalKafkaBroker
from fastkafka._testing.test_utils import mock_AIOKafkaProducer_send

In [ ]:
# | export


ProduceCallable = Callable[..., Union[Awaitable[BaseModel], BaseModel]]

In [ ]:
# | export


def _to_json_utf8(o: Any) -> bytes:
    """Converts to JSON and then encodes with UTF-8"""
    if hasattr(o, "json"):
        return o.json().encode("utf-8")  # type: ignore
    else:
        return json.dumps(o).encode("utf-8")

In [ ]:
assert _to_json_utf8({"a": 1, "b": [2, 3]}) == b'{"a": 1, "b": [2, 3]}'


class A(BaseModel):
    name: str = Field()
    age: int


assert _to_json_utf8(A(name="Davor", age=12)) == b'{"name": "Davor", "age": 12}'

In [ ]:
# | export


Message = namedtuple("Message", "message, key", defaults=[None])

In [ ]:
message = Message("Some message")
assert message.message == "Some message"
assert message.key == None

message = Message("Some message", b"123")
assert message.message == "Some message"
assert message.key == b"123"

In [ ]:
# | export


def _wrap_in_message(message: Union[BaseModel, Message]) -> Message:
    return message if type(message) == Message else Message(message)

In [ ]:
message = A(name="Davor", age=12)
wrapped = _wrap_in_message(message)

assert type(wrapped) == Message
assert wrapped.message == message
assert wrapped.key == None

In [ ]:
message = Message(A(name="Davor", age=12), b"123")
wrapped = _wrap_in_message(message)

assert type(wrapped) == Message
assert wrapped.message == message.message
assert wrapped.key == b"123"

In [ ]:
# | export


def producer_decorator(
    producer_store: Dict[str, Any], func: ProduceCallable, topic: str
) -> ProduceCallable:
    """todo: write documentation"""
    print(f"producer_decorator: {producer_store=}")
    @functools.wraps(func)
    async def _produce_async(
        *args: List[Any], producer_store=producer_store, **kwargs: Any
    ) -> BaseModel:
        f: Callable[..., Awaitable[BaseModel]] = func  # type: ignore
        return_val = await f(*args, **kwargs)
        wrapped_val = _wrap_in_message(return_val)
        _, producer, _ = producer_store[topic]
        fut = await producer.send(
            topic, _to_json_utf8(wrapped_val.message), key=wrapped_val.key
        )
        msg = await fut
        return return_val

    @functools.wraps(func)
    def _produce_sync(
        *args: List[Any], producer_store=producer_store, **kwargs: Dict[str, Any]
    ) -> BaseModel:
        f: Callable[..., BaseModel] = func  # type: ignore
        return_val = f(*args, **kwargs)
        wrapped_val = _wrap_in_message(return_val)
        _, producer, _ = producer_store[topic]
        producer.send(topic, _to_json_utf8(wrapped_val.message), key=wrapped_val.key)
        return return_val

    return _produce_async if iscoroutinefunction(func) else _produce_sync  # type: ignore

In [ ]:
async def test_me(is_async: bool):
    with mock_AIOKafkaProducer_send() as send_mock:
        topic = "test_topic"

        class MockMsg(BaseModel):
            name: str = "Micky Mouse"
            id: int = 123

        if is_async:

            async def func(mock_msg: MockMsg) -> MockMsg:
                return mock_msg

        else:

            def func(mock_msg: MockMsg) -> MockMsg:
                return mock_msg

        async with LocalKafkaBroker() as bootstrap_server:
            producer = AIOKafkaProducer(bootstrap_servers=bootstrap_server)
            if not is_async:
                producer = AIOKafkaProducerManager(producer)

            await producer.start()
            try:
                test_func = producer_decorator({topic: (None, producer, None)}, func, topic)
                assert iscoroutinefunction(test_func) == is_async

                mock_msg = MockMsg()
                if not is_async:
                    value = test_func(mock_msg)
                    await asyncio.sleep(1)
                else:
                    value = await test_func(mock_msg)

                send_mock.assert_called_once_with(
                    topic, mock_msg.json().encode("utf-8"), key=None
                )
                assert value == mock_msg

            finally:
                await producer.stop()


for is_async in [True, False]:
    await test_me(is_async)

print("ok")

[INFO] fastkafka._components.helpers: Java is already installed.
[INFO] fastkafka._components.helpers: But not exported to PATH, exporting...
[INFO] fastkafka._components.helpers: Kafka is installed.
[INFO] fastkafka._components.helpers: But not exported to PATH, exporting...
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[WARNING] aiokafka.cluster: No broker metadata found in MetadataResponse
{'test_topic': (None, <aiokafka.producer.producer.AIOKafkaProducer object>, None)}
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 162436...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 162436 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 162075...
[INFO] fastkafka._components._subprocess: termi